# slack_actions

In [ ]:
#| default_exp slack_actions

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from tk_slack.core import ValueFormatter
from tk_slack.snowflake_connector import SnowflakeConnector

from fastcore.basics import patch_to
from fastcore.test import *


from typing import List, Tuple, Dict, Any, Callable, Optional

import pandas as pd
import json, re, os
from datetime import datetime
import pytz
import time
import random
import uuid

# ActionHandler

In [ ]:
#| export

class ActionHandler:
    """
    Handles Slack interactive actions, including acknowledgment, response, 
    and storing results in Snowflake.
    
    Implements a singleton pattern to ensure only one handler exists.
    """
    
    # Singleton instance
    _instance = None
    snowflake = SnowflakeConnector()
    @classmethod
    def get_instance(cls):
        """Get or create the singleton instance.
            
        Returns:
            ActionHandler instance
        """
        if cls._instance is None:
            cls._instance = ActionHandler()
        return cls._instance
    
    def __init__(self):
        """Initialize with a Snowflake connector.
        
        Args:
            snowflake_connector: Object with methods to interact with Snowflake
        """
        if ActionHandler._instance is not None:
            raise Exception("This class is a singleton. Use get_instance() instead.")
            
        ActionHandler._instance = self
    
    @staticmethod
    def get_payload_from_body(body: Dict[str, Any]) -> Dict[str, Any]:
        """Extract relevant payload information from a Slack action body.
        
        Args:
            body: Slack action body
            
        Returns:
            Dictionary with standardized payload information
        """
        action = body["actions"][0]
        action_id = action["action_id"]
        print(f'Actiob Body: {body}')
        print(f'Action ID: {action_id}')
        print(f'Action: {action}')
        # Get common fields
        payload = {
            "user_id": body["user"]["id"],
            "user_name": body["user"]["name"],
            "channel_id": body["channel"]["id"],
            "message_ts": body["message"]["ts"],
            "action_id": action_id,
            "action_type": action["type"],
            "response_url": body["response_url"]
        }
        
        # Add type-specific data
        if action["type"] == "button":
            payload["value"] = action.get("value", "")
            payload["text"] = action["text"]["text"]
            
        elif action["type"] == "static_select" or action["type"] == "multi_static_select":
            if "selected_option" in action:
                payload["value"] = action["selected_option"]["value"]
                payload["text"] = action["selected_option"]["text"]["text"]
            elif "selected_options" in action:
                payload["values"] = [option["value"] for option in action["selected_options"]]
                payload["texts"] = [option["text"]["text"] for option in action["selected_options"]]
                
        elif action["type"] == "datepicker":
            payload["selected_date"] = action.get("selected_date", "")
            
        elif action["type"] == "users_select":
            payload["selected_user"] = action.get("selected_user", "")
            
        elif action["type"] == "channels_select":
            payload["selected_channel"] = action.get("selected_channel", "")
            
        return payload

### Responding to Actions:

In [ ]:
#| export

@patch_to(ActionHandler,cls_method=True)
def _format_response_text(self, template: str, action_data: Dict[str, Any]) -> str:
    """Format response text by replacing placeholders.
    
    Args:
        template: Response text template
        action_data: Action data with values
        
    Returns:
        Formatted response text
    """
    # Make a copy that we'll modify
    result = template
    
    # Replace standard placeholders
    result = result.replace("{user}", action_data["user_name"])
    result = result.replace("{user_id}", action_data["user_id"])
    result = result.replace("{channel}", action_data["channel_id"])
    
    # Action-specific replacements
    if "value" in action_data:
        result = result.replace("{value}", str(action_data["value"]))
    
    if "text" in action_data:
        result = result.replace("{text}", str(action_data["text"]))
        
    if "selected_date" in action_data:
        date_str = action_data["selected_date"]
        try:
            # Try to format the date nicely
            date_obj = datetime.strptime(date_str, "%Y-%m-%d")
            formatted_date = date_obj.strftime("%B %d, %Y")
            result = result.replace("{date}", formatted_date)
        except:
            # Fallback to raw date
            result = result.replace("{date}", date_str)
            
    if "selected_user" in action_data:
        result = result.replace("{selected_user}", str(action_data["selected_user"]))
        
    if "selected_channel" in action_data:
        result = result.replace("{selected_channel}", str(action_data["selected_channel"]))
        
    # For multi-select values, join them
    if "values" in action_data:
        values_str = ", ".join(str(v) for v in action_data["values"])
        result = result.replace("{values}", values_str)
        
    if "texts" in action_data:
        texts_str = ", ".join(str(t) for t in action_data["texts"])
        result = result.replace("{texts}", texts_str)    
    return result

In [ ]:
#| export

@patch_to(ActionHandler,cls_method=True)
def _send_response(self, body, action_data: Dict[str, Any], respond):
    """Send an appropriate response based on the action data.
    
    Args:
        action_data: Processed action data
        respond: Slack respond function or response URL
    """
    
    response_text = body['message']['metadata'].get('event_payload', {}).get("response_message", "Thank you for your response!")
    response_type = body['message']['metadata'].get('event_payload', {}).get("response_type", "ephemeral")
    fmt_text = self._format_response_text(response_text, action_data)
    
    # Prepare response payload
    response_payload = {
        "text": fmt_text,
        "replace_original": body['message']['metadata'].get('event_payload', {}).get("replace_original", False),
        "response_type": response_type
        }
    if callable(respond): respond(**response_payload)

    return response_payload

### Storing Action in Snowflake

When the user interacts with the bot, the action is stored in Snowflake. The action is stored in a table called `user_interaction`. The table has the following columns:

In [ ]:
#| export

@patch_to(ActionHandler,cls_method=True)
def _store_action_in_snowflake(self, action_data: Dict[str, Any],
                                table_name: str = "SLACK_INTERACTIONS"):
    """Store interaction data in Snowflake.
    
    Args:
        action_data: Action data to store
        view_info: View information extracted from metadata
        table_name: Snowflake table name
    """
    if not self.snowflake: self.snowflake = SnowflakeConnector()
            
    try:
        # Convert to format suitable for Snowflake
        view_info = action_data.get("view_info", {})
        snowflake_data = {
            "ACTION_ID": action_data["action_id"],
            "VIEW": view_info.get("view",view_info.get("view_name","unknown")),
            "VIEW_GROUP": view_info.get("view_group", "unknown"),
            "ACTION_TYPE": action_data['action_type'],
            "ACTION_INDEX": action_data['action_index'],
            "ACTION_METADATA": json.dumps(action_data['metadata']),
            "USER_ID": action_data["user_id"],
            "USER_NAME": action_data["user_name"],
            "CHANNEL_ID": action_data["channel_id"],
            "MESSAGE_TS": action_data["message_ts"],
            "RESPONSE_VALUE": action_data.get("value", 
                            action_data.get("selected_date",
                            action_data.get("selected_user",
                            action_data.get("selected_channel", "")))),
            "RESPONSE_TEXT": action_data.get("text", ""),
            "TIMESTAMP": datetime.now().isoformat(),
            "RAW_PAYLOAD": json.dumps(action_data)
        }
        
        # If we have multiple values (from multi-select), store as JSON
        if "values" in action_data:
            snowflake_data["RESPONSE_VALUES"] = json.dumps(action_data["values"])
            snowflake_data["RESPONSE_TEXTS"] = json.dumps(action_data["texts"])
        
        # Use the connector to insert into Snowflake
        self.snowflake.insert_record(table_name, snowflake_data)
    except Exception as e:
        print(f"Error storing action in Snowflake: {e}")

We also need a way to get information about a view in Snowflake:

# ActionIdManager

In [ ]:
#| export

class ActionIdManager:
    """
    Manages action IDs for Slack interactive elements to ensure uniqueness
    and provide routing capabilities for action handlers.
    """
    
    # Format: tk_interaction_type_index
    # Example: tk_interaction_btn_0
    ACTION_ID_PATTERN = "tk_interaction_{type}_{idx}"
    
    # For parsing action IDs back into components
    ACTION_ID_REGEX = re.compile(r"tk_interaction_(?P<type>[^_]+)_(?P<idx>\d+)?")
    
    @classmethod
    def generate_action_id(cls, action_type: str, index: int = 0, *args, **kwargs) -> str:
        """Generate a compact, unique action ID for a Slack interactive element.
        
        Args:
            action_type: Type of action (e.g., 'button', 'datepicker')
            index: Index for multiple elements of same type
            metadata: Optional metadata dictionary for action handling
            
        Returns:
            Unique action ID string
        """
        # Abbreviate common action types
        type_abbr = {
            'button': 'btn',
            'datepicker': 'date',
            'static_select': 'sel',
            'multi_static_select': 'msel',
            'users_select': 'user',
            'channels_select': 'chan'
            }.get(action_type, action_type[:4])  # Use first 4 chars for other types
        
        return f"tk_interaction_{type_abbr}_{index}"
    
    @classmethod
    def parse_action_id(cls, action_id: str) -> Dict[str, Any]:
        """Parse an action ID back into its components.
        
        Args:
            action_id: Action ID to parse
            
        Returns:
            Dictionary with action components and metadata
        """
        match = cls.ACTION_ID_REGEX.match(action_id)
        if not match:
            return {
                "action_type": None,
                "index": 0,
                "metadata": None
            }
        
        components = match.groupdict()
        
        # Map abbreviated types back to full names
        type_map = {
            'btn': 'button',
            'date': 'datepicker',
            'sel': 'static_select',
            'msel': 'multi_static_select',
            'user': 'users_select',
            'chan': 'channels_select'
        }
        
        # Set action type from mapping or keep as is
        components["action_type"] = type_map.get(components["type"], components["type"])
        components["index"] = int(components["idx"])
            
        return components

In [ ]:
test_eq(ActionIdManager.generate_action_id("button", 0), "tk_interaction_btn_0")

test_eq(ActionIdManager.parse_action_id("tk_interaction_btn_0"),{'type': 'btn', 'idx': '0', 'action_type': 'button', 'index': 0})


Let's now update the `ActionHandler` class to use the new `ActionIDManager` class.

Let's create a method to allow the user to setup the action handler in their slackbot app, and this handler will need to use an instance of the `ActionIDManager` class to generate a unique action ID for each action. The action ID will be used to identify the action in the Snowflake table

In [ ]:
#| export

@patch_to(ActionHandler,cls_method=True)
def setup_slack_action_handler(self, app):
    """Set up a single Slack action handler with the Bolt app.
    
    Args:
        app: Slack Bolt app
        snowflake_connector: Connector for Snowflake operations
        
    Returns:
        Initialized ActionHandler instance
    """
    # Create the singleton action handler
    handler = ActionHandler.get_instance()
    ACTION_ID_PREFIX_REGEX = re.compile(r"tk_interaction_(?P<type>[^_]+)_(?P<idx>\d+)?")
    
    # Register the catch-all action handler
    @app.action(ACTION_ID_PREFIX_REGEX)
    def handle_all_actions(ack, body, logger, respond):
        """Universal handler for all interactive elements."""
        # Always acknowledge receipt
        ack()
        
        try:
            # Validate body structure
            if not body:
                logger.error("Body is None")
                return
                
            if 'actions' not in body or not body['actions']:
                logger.error(f"No actions in body: {json.dumps(body)}")
                return
                            
            # Process the action
            if handler is None:
                logger.error("Handler is None")
                return
                
            handler.process_slack_action(body, respond)
            
        except Exception as e:
            logger.error(f"Error processing action: {str(e)}")
            logger.error(f"Action body: {json.dumps(body) if body else 'None'}")
            import traceback
            logger.error(f"Traceback: {traceback.format_exc()}")
    
    print("Registered universal action handler for all block actions")
    
    return handler

We also need to update the `ActionHandler` class to use the new `ActionIDManager` class. When processing actions from the app, it will need to call `parse_action_id` to get the information from the id

In [ ]:
body_example = {'type': 'block_actions', 'user': {'id': 'U08SWJ7MGDC', 'username': 'cooperrichason', 'name': 'cooperrichason', 'team_id': 'T08SWJ7MGAJ'}, 'api_app_id': 'A08SWJJGW1L', 'token': 'wjBURx2qV2cSMgnUlvDvI7D9', 'container': {'type': 'message', 'message_ts': '1747869345.068909', 'channel_id': 'C08TFTZHY4R', 'is_ephemeral': False}, 'trigger_id': '8931953858965.8914619730358.3acc5cd59b50a162f407f5d3a049a08b', 'team': {'id': 'T08SWJ7MGAJ', 'domain': 'datatistics'}, 'enterprise': None, 'is_enterprise_install': False, 'channel': {'id': 'C08TFTZHY4R', 'name': 'tk_slack'}, 'message': {'user': 'U08TFUY0HND', 'type': 'message', 'ts': '1747869345.068909', 'bot_id': 'B08TFUXV0M7', 'app_id': 'A08SWJJGW1L', 'text': 'Example Data Alert', 'team': 'T08SWJ7MGAJ', 'metadata': {'event_type': 'example_view_notification', 'event_payload': {'view_name': 'example_view', 'view_group': 'examples', 'response_type': 'ephemeral', 'response_message': 'Thanks {user}! You selected "{text}" ({value})', 'replace_original': False, 'custom_row_index': 2}}, 'blocks': [{'type': 'section', 'block_id': 'DpeTS', 'text': {'type': 'mrkdwn', 'text': '*Feature Z Implementation*', 'verbatim': False}}, {'type': 'section', 'block_id': 'BOqKS', 'text': {'type': 'mrkdwn', 'text': 'Planning for Feature Z implementation', 'verbatim': False}}, {'type': 'section', 'block_id': '3rZsp', 'fields': [{'type': 'mrkdwn', 'text': '*Name*\nFeature Z Implementation', 'verbatim': False}, {'type': 'mrkdwn', 'text': '*Text*\nPlanning for Feature Z implementation', 'verbatim': False}, {'type': 'mrkdwn', 'text': '*Status*\nIn Progress', 'verbatim': False}, {'type': 'mrkdwn', 'text': '*Priority*\nHigh', 'verbatim': False}, {'type': 'mrkdwn', 'text': '*Due Date*\nJun 15, 2025', 'verbatim': False}, {'type': 'mrkdwn', 'text': '*Option Name*\nStart Work, Assign to Me, Request More Info', 'verbatim': False}, {'type': 'mrkdwn', 'text': '*Option Value*\nstart, assign_self, request_info', 'verbatim': False}]}, {'type': 'actions', 'block_id': 'WKbX2', 'elements': [{'type': 'button', 'action_id': 'tk_interaction_btn_0', 'text': {'type': 'plain_text', 'text': 'Start Work', 'emoji': True}, 'style': 'primary', 'value': 'start'}, {'type': 'button', 'action_id': 'tk_interaction_btn_1', 'text': {'type': 'plain_text', 'text': 'Assign to Me', 'emoji': True}, 'value': 'assign_self'}, {'type': 'button', 'action_id': 'tk_interaction_btn_2', 'text': {'type': 'plain_text', 'text': 'Request More Info', 'emoji': True}, 'value': 'request_info'}]}, {'type': 'divider', 'block_id': 'MnqyR'}]}, 'state': {'values': {}}, 'response_url': 'https://hooks.slack.com/actions/T08SWJ7MGAJ/8931953776821/BeYdqU9hrCKACdosBdn9GVBA', 'actions': [{'action_id': 'tk_interaction_btn_0', 'block_id': 'WKbX2', 'text': {'type': 'plain_text', 'text': 'Start Work', 'emoji': True}, 'value': 'start', 'style': 'primary', 'type': 'button', 'action_ts': '1747870391.878433'}]}
body_example

{'type': 'block_actions',
 'user': {'id': 'U08SWJ7MGDC',
  'username': 'cooperrichason',
  'name': 'cooperrichason',
  'team_id': 'T08SWJ7MGAJ'},
 'api_app_id': 'A08SWJJGW1L',
 'token': 'wjBURx2qV2cSMgnUlvDvI7D9',
 'container': {'type': 'message',
  'message_ts': '1747869345.068909',
  'channel_id': 'C08TFTZHY4R',
  'is_ephemeral': False},
 'trigger_id': '8931953858965.8914619730358.3acc5cd59b50a162f407f5d3a049a08b',
 'team': {'id': 'T08SWJ7MGAJ', 'domain': 'datatistics'},
 'enterprise': None,
 'is_enterprise_install': False,
 'channel': {'id': 'C08TFTZHY4R', 'name': 'tk_slack'},
 'message': {'user': 'U08TFUY0HND',
  'type': 'message',
  'ts': '1747869345.068909',
  'bot_id': 'B08TFUXV0M7',
  'app_id': 'A08SWJJGW1L',
  'text': 'Example Data Alert',
  'team': 'T08SWJ7MGAJ',
  'metadata': {'event_type': 'example_view_notification',
   'event_payload': {'view_name': 'example_view',
    'view_group': 'examples',
    'response_type': 'ephemeral',
    'response_message': 'Thanks {user}! You 

In [ ]:
#| export

@patch_to(ActionHandler,cls_method=True)
def process_slack_action(self, 
                   body: Dict[str, Any], 
                   respond: Callable):
    """Process a Slack action event.
    
    Args:
        body: Slack event body
        respond: Slack respond function

    Returns:
        Processed action data
    """
    print(body)
    action_data = {}
    action_data['action_id'] = body['actions'][0]['action_id']
    action_data['action_type'] = body['actions'][0]['type']
    action_data['user_id'] = body['user']['id']
    action_data['user_name'] = body['user']['name']
    action_data['action_index'] = body['actions'][0]['action_id'].split("_")[3]
    action_data['channel_id'] = body['channel']['id']
    action_data['message_ts'] = body['message']['ts']
    action_data['value'] = body['actions'][0].get('value', '')
    action_data['selected_date'] = body['actions'][0].get('selected_date', '')
    action_data['selected_user'] = body['actions'][0].get('selected_user', '')
    action_data['selected_channel'] = body['actions'][0].get('selected_channel', '')
    action_data['metadata'] = body['actions'][0].get('metadata', {})
    action_data['view_info'] = body['message']['metadata'].get('event_payload', {})

    # Store interaction in Snowflake
    responce_payload = self._send_response(body, action_data, respond)

    action_data['text'] = responce_payload['text']

    self._store_action_in_snowflake(action_data = action_data)
    
    return action_data

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()